In [1]:
from datasets import load_dataset
from PIL import Image
from datasets import load_dataset, load_from_disk
import time
import torch
from torch.utils.data import DataLoader, random_split
from torch import nn
from torchvision import transforms
import numpy as np
import os
from pathlib import Path
import PIL
from sklearn.model_selection import train_test_split
batch_size = 8

/Users/sphemister/Documents/Github/more-buda-demos/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sphemister/Documents/Github/more-buda-demos/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
try:
    first_time = False
    ds = load_from_disk(str(Path(os.path.abspath('.')) / '..' / 'image'/'cats_vs_dogs_tensor'))
except:
    first_time = True
    ds = load_dataset("microsoft/cats_vs_dogs", split='train')
id_to_label_name = ['cat','dog']

In [ ]:
unique_datatypes = set(str(type(feature)) for feature in ds['image'])

In [9]:
unique_datatypes

{"<class 'PIL.Image.Image'>", "<class 'PIL.JpegImagePlugin.JpegImageFile'>"}

In [7]:
if first_time:
    ds = ds.select(range(800))
    def filter_by_image_size(image):
        min_width = 200
        min_height = 200
        max_width = 512
        max_height = 512
        width, height = image.size
        return min_width <= width <= max_width and min_height <= height <= max_height

    torch_preprocessing = transforms.Compose([
        transforms.ColorJitter(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation((-10,10)),
        transforms.Resize((256,256)),
    ])
    def transform_mapper(sample):
        # image = sample['image'].convert("RGB")
        sample['image'] = torch_preprocessing(sample['image'])
        return sample
    
    filtered_dataset = ds.filter(lambda example: filter_by_image_size(example['image']))
    transformed_dataset = filtered_dataset.map(transform_mapper, batched=False)
    def cast_to_float32(batch):
        batch['image'] = batch['image'].to(torch.float32)
        batch['labels'] = batch['labels']
        return batch
    transformed_dataset = transformed_dataset.map(cast_to_float32)
    # save the imageset
    transformed_dataset.save_to_disk(str(Path(os.path.abspath('.')) / '..' / 'image'/'cats_vs_dogs_tensor'))
else:
    transformed_dataset = ds

In [8]:
# doesnt save the dataset format to disk in this format
transformed_dataset.set_format(type='torch', columns=['image','labels'])

In [16]:
ds_train, ds_test = random_split(transformed_dataset, [700, 37])

In [17]:
cats_dogs_dl = DataLoader(ds_train, batch_size=8, shuffle=True, drop_last=True, )
cats_dogs_test = DataLoader(ds_test, batch_size=8, shuffle=True, drop_last=True, )

In [12]:
data = next(iter(cats_dogs_dl))

In [13]:
data['image']

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
        

In [14]:
data['image'].shape

torch.Size([8, 3, 256, 256])

In [15]:
data['labels']

tensor([0, 0, 0, 0, 0, 0, 0, 0])

## Import the model

In [24]:
from models.cat_dog_torch import Cat_Dog_CNN
activation_func = nn.ReLU()
model = Cat_Dog_CNN(in_channels=3,out_channels=3,num_conv_layers=5,activation_function=activation_func, normalization=True, num_classifications=2)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1E-3, weight_decay=1E-8)

In [19]:
epochs = 10

In [27]:

model.train()
training_accuracy = []
training_loss = []
test_accuracy = []
test_loss = []
epochs = 10
epsilon = 1E-6
num_samples = len(cats_dogs_dl.dataset)
num_samples -= num_samples%8

num_test_samples = len(cats_dogs_test.dataset)
num_test_samples -= num_test_samples%8

start_time = time.perf_counter()

for epoch in range(1,epochs+1):
    print('Epoch #',epoch)
    
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0
    for batch_idx, data in enumerate(cats_dogs_dl):
        # training
        batch_size = data['image'].size(0)
        optimizer.zero_grad()
        output = model(data['image'])
        # print('output shape: ', output.shape)
        # print('output: ', )
        # print('target shape: ', data['labels'].shape)
        # print('one hot target: ', nn.functional.one_hot(data['labels']).shape)
        loss = loss_func(output, data['labels'])
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        # accuracy
        preds = torch.argmax(output, dim=1)
        epoch_accuracy += preds.eq(data['labels']).sum().item()
    
    epoch_loss /= num_samples
    training_loss.append(epoch_loss)
    
    epoch_accuracy /= num_samples
    training_accuracy.append(epoch_accuracy)
    print('Training Loss: ', epoch_loss, ' Accuracy: ', epoch_accuracy)
    
    model.eval()
    epoch_test_accuracy = 0
    for data in cats_dogs_test:
        
        with torch.no_grad():
            output = model(data['image'])
            preds = torch.argmax(output, dim=1)
            epoch_test_accuracy+= preds.eq(data['labels']).sum().item()
    epoch_test_accuracy /= num_test_samples
    test_accuracy.append(epoch_test_accuracy)
    print('Test Accuracy: ', test_accuracy[-1])
    
    # epsilon breaking - avoids overfitting
    if len(training_loss) > 1 and abs(training_loss[-1] - training_loss[-2])  < epsilon:
        break

end_time = time.perf_counter()

total_time = end_time - start_time
print(f'Training Took {total_time:.1f} seconds')    

Epoch # 1
Training Loss:  0.03956934424309895  Accuracy:  1.0
Test Accuracy:  1.0
Epoch # 2
Training Loss:  0.03927129622677277  Accuracy:  1.0
Test Accuracy:  1.0
Epoch # 3
Training Loss:  0.03921694552590107  Accuracy:  1.0
Test Accuracy:  1.0
Epoch # 4
Training Loss:  0.03919401255318488  Accuracy:  1.0
Test Accuracy:  1.0
Epoch # 5
Training Loss:  0.03918208742792579  Accuracy:  1.0
Test Accuracy:  1.0
Epoch # 6
Training Loss:  0.039175304916055725  Accuracy:  1.0
Test Accuracy:  1.0
Epoch # 7
Training Loss:  0.03917103629002626  Accuracy:  1.0
Test Accuracy:  1.0
Epoch # 8
Training Loss:  0.03916785279395937  Accuracy:  1.0
Test Accuracy:  1.0
Epoch # 9
Training Loss:  0.03916582717806443  Accuracy:  1.0
Test Accuracy:  1.0
Epoch # 10
Training Loss:  0.039164359499325695  Accuracy:  1.0
Test Accuracy:  1.0
Training Took 97.2 seconds
